In [3]:
import sys
!{sys.executable} -m pip -q install pandas
!{sys.executable} -m pip -q install geopandas
!{sys.executable} -m pip -q install folium
!{sys.executable} -m pip -q install ipyleaflet
!{sys.executable} -m pip -q install matplotlib
!{sys.executable} -m pip -q install geopy



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Read in CITI Bike Station Locations

In [118]:
import json
bikeStationsFilepath = "./input/citi_bike_station_information.json"
fp = open(bikeStationsFilepath, "r")
stationInformation = json.load(fp)
bikeStations = stationInformation["data"]["stations"]

# NYC Income Data

In [1]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

In [2]:
boroughCol = "borough"
neighborhoodsGeometryFilepath = "./input/nyc_neighborhoods.geojson"
dfNeighborhoods = gpd.read_file(neighborhoodsGeometryFilepath)
manhattanFilter = dfNeighborhoods[boroughCol] == "Manhattan"
dfManhattan = dfNeighborhoods[manhattanFilter]

Differing neighborhood granularity amongst income data and GeoJSON data. Normalize the neighborhoods with manual mappings?

# Subway Stops

In [22]:
import pandas as pd
from geopy.distance import distance
from geopy.geocoders import Nominatim
import numpy as np
import time

In [9]:
trainStationsFilepath = "./input/subway_stops.txt"
trainStopIdCol = "stop_id"
trainStopLatCol = "stop_lat"
trainStopLongCol = "stop_lon"
bikeStationIdCol = "station_id"
bikeStationLatitudeCol = "lat"
bikeStationLongitudeCol = "lon"
addressField = "address"
neighborhoodField = "neighbourhood"
boroughField = "borough"
trainStopsAugmentedFilepath = "./input/subway_stops_augmented.csv"
bikeStationsAugmentedFilepath = "./input/bike_stations_augmented.csv"
dfTrainStops = pd.read_csv(trainStationsFilepath)
dfBikeStations = pd.DataFrame(columns = [bikeStationIdCol, bikeStationLatitudeCol, bikeStationLongitudeCol])
#Remove duplicate stops
dfTrainStopsFiltered = dfTrainStops[~((dfTrainStops[trainStopIdCol].str.endswith("N")) | (dfTrainStops[trainStopIdCol].str.endswith("S")))]

Geocode train station data

In [ ]:
def reverse_geocode_coordinates(latitude, longitude):
    neighborhoodGeojsonField = "neighborhood"
    geometryField = "geometry"
    neighborhood = "None"
    borough = "None"
    for index, row in dfNeighborhoods.iterrows():
        polygon = row[geometryField]
        point = Point(longitude, latitude)
        if point.within(polygon):
           neighborhood = row[neighborhoodGeojsonField]
           borough = row[boroughField]
           break
    return {neighborhoodField: neighborhood, boroughCol: borough}

In [ ]:
boroughVals = list()
neighborhoodVals = list()
for index, row in dfTrainStopsFiltered.iterrows():
    #Get location data and extract neighborhood and borough for the provided coordinates
    locationData = reverse_geocode_coordinates(row[trainStopLatCol], row[trainStopLongCol])
    neighborhoodVals.append(locationData[neighborhoodField])
    boroughVals.append(locationData[boroughField])
dfTrainStopsFiltered[neighborhoodField] = neighborhoodVals
dfTrainStopsFiltered[boroughField] = boroughVals
#Save reverse geocoded data to file
dfTrainStopsFiltered.to_csv(trainStopsAugmentedFilepath, index = False)

/var/folders/33/0fb406ns7l1dyhs68vq615540000gp/T/ipykernel_4669/318852425.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrainStopsFiltered[neighborhoodField] = neighborhoodVals
/var/folders/33/0fb406ns7l1dyhs68vq615540000gp/T/ipykernel_4669/318852425.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrainStopsFiltered[boroughField] = boroughVals


Augment Citi bike stations with neighborhood and borough information

In [123]:
#Extract latitude and longitude for bike stations and store in dataframe
for stationObj in bikeStations:
    rowVals = [stationObj[bikeStationIdCol], stationObj[bikeStationLatitudeCol], stationObj[bikeStationLongitudeCol]]
    dfBikeStations.loc[len(dfBikeStations)] = rowVals

In [125]:
boroughVals = list()
neighborhoodVals = list()
for index, row in dfBikeStations.iterrows():
    #Get location data and extract neighborhood and borough for the provided coordinates
    locationData = reverse_geocode_coordinates(row[bikeStationLatitudeCol], row[bikeStationLongitudeCol])
    neighborhoodVals.append(locationData[neighborhoodField])
    boroughVals.append(locationData[boroughField])
dfBikeStations[neighborhoodField] = neighborhoodVals
dfBikeStations[boroughField] = boroughVals
#Save reverse geocoded data to file
dfBikeStations.to_csv(bikeStationsAugmentedFilepath, index = False)

Compute distance to closest bike station for each train station

In [158]:
def closest_bike_station_distance(trainStopCoord, dfBikeStations):
    #Initialize variable to track the distance from the train stop to the closest bike station
    closestBikeStationDistance = np.inf
    #Find the closest bike station out of all the possible bike stations
    for index, row in dfBikeStations.iterrows():
        bikeStationCoord = (row[bikeStationLatitudeCol], row[bikeStationLongitudeCol])
        #Compute distance between train stop and bike station using GeoPy package
        bikeStationDistance = distance(trainStopCoord, bikeStationCoord).km
        if bikeStationDistance < closestBikeStationDistance:
            closestBikeStationDistance = bikeStationDistance
    return closestBikeStationDistance

In [159]:
#For each train stop, compute the distance and append this to dataframe for train stops
distanceCol = "distance_to_closest_bike_station"
dfTrainStopsAugmented = pd.read_csv(trainStopsAugmentedFilepath)
dfBikeStationsAugmented = pd.read_csv(bikeStationsAugmentedFilepath)
distanceVals = list()
for index, row in dfTrainStopsAugmented.iterrows():
    trainStopCoord = (row[trainStopLatCol], row[trainStopLongCol])
    distanceVals.append(closest_bike_station_distance(trainStopCoord, dfBikeStations))
dfTrainStopsAugmented[distanceCol] = distanceVals
dfTrainStopsAugmented.to_csv(trainStopsAugmentedFilepath, index = False)

Instead of making a table grouped by subway line, make a table grouped by borough or neighborhood (use lat/long of station to geocode)

Number of train stations, number of Citi bike stations, average distance from train station to closest bike station

In [7]:
import pandas as pd

In [11]:
dfBikeStationsAugmented = pd.read_csv(bikeStationsAugmentedFilepath)
dfBikeStationsGrouped = dfBikeStationsAugmented.groupby(neighborhoodField).agg({bikeStationIdCol:"count"}).reset_index()
dfBikeStationsGrouped = dfBikeStationsGrouped.rename(columns = {"station_id":"bike_station_count"})
dfBikeStationsGrouped

,neighbourhood,bike_station_count
0,Astoria,49
1,Battery Park City,7
2,Bay Ridge,6
3,Bedford-Stuyvesant,83
4,Belmont,14
...,...,...
99,West Farms,9
100,West Village,21
101,Williamsburg,80
102,Windsor Terrace,12


In [13]:
distanceCol = "distance_to_closest_bike_station"
dfTrainStopsAugmented = pd.read_csv(trainStopsAugmentedFilepath)
dfTrainStopsGrouped = dfTrainStopsAugmented.groupby(neighborhoodField).agg({trainStopIdCol:"count", distanceCol:"mean"}).reset_index()
dfTrainStopsGrouped = dfTrainStopsGrouped.rename(columns = {"stop_id":"train_station_count", "distance_to_closest_bike_station":"average_distance_to_bike_station"})
dfTrainStopsGrouped["average_distance_to_bike_station"] = round(dfTrainStopsGrouped["average_distance_to_bike_station"], 3)
dfTrainStopsGrouped

,neighbourhood,train_station_count,average_distance_to_bike_station
0,Allerton,2,1.704
1,Arverne,2,13.122
2,Astoria,4,0.082
3,Bath Beach,1,3.560
4,Bay Ridge,2,0.530
...,...,...,...
128,Williamsburg,12,0.078
129,Windsor Terrace,2,0.093
130,Woodhaven,3,2.598
131,Woodrow,2,17.565


In [19]:
dfResult = dfTrainStopsGrouped.merge(dfBikeStationsGrouped, left_on = neighborhoodField, right_on = neighborhoodField, how = "inner")
dfResult = dfResult.loc[:, ["neighbourhood", "train_station_count", "bike_station_count", "average_distance_to_bike_station"]]
dfResult = dfResult.rename(columns = {"neighbourhood":"Neighborhood", "train_station_count":"Number of Train Stations", "bike_station_count":"Number of Bike Stations", "average_distance_to_bike_station":"Mean Distance from Train Stop to Bike Station (km)"})
dfResult

,Neighborhood,Number of Train Stations,Number of Bike Stations,Mean Distance from Train Stop to Bike Station (km)
0,Astoria,4,49,0.082
1,Bay Ridge,2,6,0.530
2,Bedford-Stuyvesant,10,83,0.109
3,Boerum Hill,1,10,0.023
4,Bronx Park,1,1,0.942
...,...,...,...,...
73,West Farms,1,9,0.058
74,West Village,4,21,0.033
75,Williamsburg,12,80,0.078
76,Windsor Terrace,2,12,0.093


In [20]:
neighborhoodStatsFilepath = "./output/neighborhood_stats.csv"
dfResult.to_csv(neighborhoodStatsFilepath, index = False)